In [1]:
import pandas as pd
from google import genai
from google.genai import types

In [2]:
english_definitions = pd.read_parquet("data/english_definitions.parquet")

In [4]:
english_definitions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   index       301 non-null    int64 
 1   characters  301 non-null    object
 2   pinyin      301 non-null    object
 3   definition  301 non-null    object
dtypes: int64(1), object(3)
memory usage: 9.5+ KB
